# Most synapses in and out  
This notebook demonstrates a few ways to work with the synapse tables we've generated. As a working example, it demonstrates how to find the neurons with the most input and output synapse predictions within a table. It also demonstrates how to view these cells and their synapses in neuroglancer, and points to further resources.

In [ ]:
import nglui
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

### Reading data

read in the soma table to get IDs of cells with cell bodies in volume

In [ ]:
soma_df = pd.read_csv('data/soma_valence_v185.csv')

In [ ]:
soma_df.head()

read in the synapse table 

In [ ]:
syn_df = pd.read_csv('data/pni_synapses_v185.csv')

It has 3.2 million rows

In [ ]:
print(syn_df.shape)

In [ ]:
syn_df.head()

Lets filter for IDs with somas that are neurons

In [ ]:
neuron_df=soma_df.query('cell_type == "i" | cell_type=="e"')

Use this list of IDs to filter all synapses to just those onto neurons 

In [ ]:
post_neuron_syn_df=syn_df[syn_df.post_root_id.isin(neuron_df.pt_root_id)]

951,457 of 3.5 million synapses are onto cells with somas in the volume

In [ ]:
post_neuron_syn_df.shape

## Most inputs

Lets summarize the number of inputs these cells have

In [ ]:
in_degree_df = post_neuron_syn_df.groupby('post_root_id').post_root_id.count()

In [ ]:
in_degree_df.head()

Let's find the cell with the most inputs

In [ ]:
max_input_neuron=in_degree_df.index[in_degree_df.argmax()]
print(max_input_neuron)

it has 5369 input synapses

In [ ]:
in_degree_df[max_input_neuron]

Lets filter synapses to just those onto this one neuron

In [ ]:
onto_cell_syn_df = post_neuron_syn_df.query(f'post_root_id == {max_input_neuron}').copy()

This illustrates how to setup a nglui pipeline to highlight synapses

### Visualization: most inputs

In [ ]:
from nglui.statebuilder import *
# get sources from the neuroglancer link
img_source = "precomputed://https://storage.googleapis.com/microns_public_datasets/pinky100_v0/son_of_alignment_v15_rechunked"
seg_source = "precomputed://gs://microns_public_datasets/pinky100_v185/seg"

# use nglui (www.github.com/seung-lab/NeuroglancerAnnotationUI) to setup a dataframe > neuroglancer link pipeline
img_layer = ImageLayerConfig(name='layer23',
                             source=img_source,
                             )
# we want the segmentation layer with our target neuron always on
seg_layer = SegmentationLayerConfig(name = 'seg',
                                    source = seg_source,
                                    fixed_ids=[max_input_neuron])

# need to consolidate the synapse position into a single column
onto_cell_syn_df.loc[:, 'ctr_pos']=onto_cell_syn_df.apply(lambda x: [x.ctr_pos_x_vx, x.ctr_pos_y_vx, x.ctr_pos_z_vx], axis=1).copy()

# setup a mapping rule for point annotations
syn_points = PointMapper(point_column='ctr_pos', linked_segmentation_column='pre_root_id')
# add these points linked to the segmentation layer name
syn_layer = AnnotationLayerConfig(name='synapses', mapping_rules=syn_points, linked_segmentation_layer='seg')


When you click the below link, you'll be taken to a neuroglancer visualization of the dataset.

If you are unfamiliar with basic neuroglancer visualization, you should read a guide here (https://microns-explorer.org/visualization)

There is also a fast intro (with movies!) written up here https://blog.eyewire.org/explore-cortical-neurons/.

In this view, this code has setup a new layer (synapses).  It should open with the synapse layer details on the right hand side, where you can see a list of synapses.  To learn more about how to interact with this neuroglancer link , you can watch this below 5 minute video. 

In [ ]:
from IPython.lib.display import YouTubeVideo
YouTubeVideo('YTbu6vMljuE', width=800, height=600)

setup a state builder with this layer pipeline

then render the state pipeline with the dataframe data

returning the link as an html link

In [ ]:
sb = StateBuilder([img_layer, seg_layer, syn_layer])
sb.render_state(onto_cell_syn_df, return_as='html')

## Most outputs

Lets repeat the exercise to find cells with the most outputs

In [ ]:
pre_neuron_syn_df = syn_df[syn_df.pre_root_id.isin(neuron_df.pt_root_id)]
out_degree_df = pre_neuron_syn_df.groupby('pre_root_id').post_root_id.count()


in contrast to inputs only 15,884 outputs of the

3.2 million synapses can be connected

to cells with cell bodies in volume

In [ ]:
pre_neuron_syn_df.shape

## Scatter plot: # inputs vs. # outputs

since we have the data, lets make a dataframe with all the cells in degree
and out degree

In [ ]:
in_out_df = pd.DataFrame({'in_degree': in_degree_df, 'out_degree':out_degree_df})

merge in the cell type information we have on these cells

In [ ]:
in_out_ct_df = pd.merge(in_out_df, soma_df[['cell_type','pt_root_id']], left_index=True, right_on='pt_root_id' )

you can see here that many cells have low out-degree 
as excitatory neurons in the bottom half of the dataset
have axons which leave the bottom of the volume
before branching, but there are 5 inhibitory neurons with >300 synapses

the in degree varies from a few hundred to >5000 depending on how much
dendrite is in the volume


In [ ]:
pg=sns.pairplot(x_vars=['in_degree'], y_vars=['out_degree'],
             data=in_out_ct_df, hue='cell_type', plot_kws={'alpha': 0.75},  height=5)


In [ ]:
in_out_ct_df.query('out_degree>300')

let's get all these synapses from these neurons

In [ ]:
is_from_most_out = pre_neuron_syn_df.pre_root_id.isin(in_out_ct_df.query('out_degree>300').pt_root_id.values)
most_inh_syn_out = pre_neuron_syn_df[is_from_most_out].copy()

here are our 4220 synapses

In [ ]:
most_inh_syn_out.shape

## Visualization: inhibitory cells with > 300 output synapses

lets visualize them all in neuroglancer

In [ ]:
most_inh_syn_out['ctr_pos']=most_inh_syn_out.apply(lambda x: [x.ctr_pos_x_vx, x.ctr_pos_y_vx, x.ctr_pos_z_vx], axis=1).values

seg_layer = SegmentationLayerConfig(name = 'seg',
                                    source = seg_source,
                                    selected_ids_column='pre_root_id')

# setup a mapping rule for point annotations
syn_points = PointMapper(point_column='ctr_pos', linked_segmentation_column='post_root_id')
# add these points linked to the segmentation layer name
syn_layer = AnnotationLayerConfig(name='synapses', mapping_rules=syn_points, linked_segmentation_layer='seg')

sb = StateBuilder([img_layer, seg_layer, syn_layer])
# returning the link as an html link
sb.render_state(most_inh_syn_out, return_as='html')

maybe that's a bit overwhelming... lets cut the dataframe to just the cell with the most
outputs, it's a basket cell, check out how many times it makes multiple synapses 
with the same target, you can quickly get a sense for how it distributes its 
outputs across different post-synaptic target structures.

In [ ]:
sb.render_state(most_inh_syn_out.query('pre_root_id == 648518346349539215'), return_as='html')